In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.cross_validation import KFold

import revrand.basis_functions as bf
from revrand.validation import smse, msll
from revrand.regression import learn, predict

In [3]:
# Load the data
boston = load_boston()
X = boston.data
y = boston.target - boston.target.mean()

N, D = X.shape

In [23]:
# Construct basis functions
base = bf.RandomRBF_ARD(Xdim=D, nbases=50) + bf.RandomRBF(Xdim=D, nbases=50)
init_params = [1000. * np.ones(D), 1000.]

In [24]:
# Cross val
folds = 5
av_smse = 0.
av_msll = 0.
for i, (tr_ind, ts_ind) in enumerate(KFold(len(y), n_folds=folds, shuffle=True)):
    
    # Training
    params = learn(X[tr_ind], y[tr_ind], base, init_params)
    
    # Prediction
    Ey, Vf, Vy = predict(X[ts_ind], base, *params)
    
    # Validation
    f_smse = smse(y[ts_ind], Ey)
    f_msll = msll(y[ts_ind], Ey, Vy, y[tr_ind])
    av_smse += f_smse
    av_msll += f_msll
    
    print("Fold: {}, SMSE = {}, MSLL = {}".format(i, f_smse, f_msll))
    
av_smse /= folds
av_msll /= folds

Fold: 0, SMSE = 0.11324296588632327, MSLL = -1.1999955192365723
Fold: 1, SMSE = 0.31539845857876325, MSLL = -0.607155852688254
Fold: 2, SMSE = 0.1571350685045705, MSLL = -0.9177251662903533
Fold: 3, SMSE = 0.1118159441690557, MSLL = -1.1281885011196762
Fold: 4, SMSE = 0.2815883496707695, MSLL = -0.6059645717247217


In [25]:
# Print results
print("Final: SMSE = {}, MSLL = {}".format(av_smse, av_msll))

Final: SMSE = 0.19583615736189647, MSLL = -0.8918059222119155
